In [32]:

import pandas as pd
import xlrd
import xlsxwriter

df = pd.read_excel ('D:\Momen\Chat_AHT_Conc_Workfile.xlsx', sheet_name='Sheet3')

df['dff'] = df['Terminated'] - df['Created']
df['DEV_ST_TIME']  = df['Created']
df['DEV_END_TIME']  = df['Terminated']
df['DEV_DIFF']= ""



for i in range(1, 42568):
    if df.loc[i, "agentname"]  ==  df.loc[i-1, "agentname"] :  
        if df.loc[i, "DEV_ST_TIME"] <= df.loc[i-1, "DEV_END_TIME"] :
            df.loc[i, "DEV_ST_TIME"] = df.loc[i - 1, "DEV_END_TIME"]
            if df.loc[i,"DEV_END_TIME"] < df.loc[i,"DEV_ST_TIME"]:
                df.loc[i,"DEV_END_TIME"] = df.loc[i,"DEV_ST_TIME"]



for i in range(0, 42568):
    if  df.loc[i,"DEV_END_TIME"] > df.loc[i,"DEV_ST_TIME"]:
        df.loc[i,"DEV_DIFF"] = df.loc[i,"DEV_END_TIME"] - df.loc[i,"DEV_ST_TIME"]
    else:
        df.loc[i,"DEV_DIFF"]= "00:00:00"
        
        
df['DEV_DIFF']=pd.to_timedelta(df.DEV_DIFF)      


df['DEV_DIFF']=pd.to_timedelta(df.DEV_DIFF) 
Agent_Group = df.groupby('agentname')
Agent_Series = Agent_Group['dff'].sum() / Agent_Group['DEV_DIFF'].sum()

Queue_Group = df.groupby('queue')
Queue_Series = Queue_Group['dff'].sum() / Queue_Group['DEV_DIFF'].sum() 

totalCon = df['dff'].sum() / df['DEV_DIFF'].sum() 
total = pd.Series(totalCon , index = ['grand total'])

Agent_Series = Agent_Series.append(total)
Queue_Series = Queue_Series.append(total)

writer = pd.ExcelWriter('D:\Momen\Final_Cancurrency.xlsx', engine='xlsxwriter')
Agent_Series.to_excel(writer, sheet_name='Agent_Concurrency', index=True)
Queue_Series.to_excel(writer, sheet_name='Queue_Concurrency', index=True)
writer.save()
#df['Con']